In [81]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.mlab import PCA as mlabPCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
%matplotlib inline

We've talked about Random Forests. Now it's time to build one.

Here we'll use data from Lending Club to predict the state of a loan given some information about it. You can find the dataset [here](https://www.lendingclub.com/info/download-data.action). We'll use 2015 data. ([Thinkful mirror](https://www.dropbox.com/s/m7z42lubaiory33/LoanStats3d.csv?dl=0))

In [73]:
# Replace the path with the correct path for your data.
y2015 = pd.read_csv(
    'LoanStats3d.csv',
        skipinitialspace=True,
    header=1
)

# Note the warning about dtypes.

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [74]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,...,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,A4,...,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,A4,...,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,A2,...,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,68495092,73384866.0,8650.0,8650.0,8650.0,36 months,19.89%,320.99,E,E3,...,0.0,12.0,100.0,50.0,1.0,0.0,38998.0,18926.0,2750.0,18248.0


## The Blind Approach

Now, as we've seen before, creating a model is the easy part. Let's try just using everything we've got and throwing it without much thought into a Random Forest. SKLearn requires the independent variables to be be numeric, and all we want is dummy variables so let's use `get_dummies` from Pandas to generate a dummy variable for every categorical colummn and see what happens off of this kind of naive approach.

from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)

cross_val_score(rfc, X, Y, cv=5)

Did your kernel die? My kernel died.

Guess it isn't always going to be that easy...

Can you think of what went wrong?

(You're going to have to reset your kernel and reload the column, BUT DON'T RUN THE MODEL AGAIN OR YOU'LL CRASH THE KERNEL AGAIN!)

## Data Cleaning

Well, `get_dummies` can be a very memory intensive thing, particularly if data are typed poorly. We got a warning about that earlier. Mixed data types get converted to objects, and that could create huge problems. Our dataset is about 400,000 rows. If there's a bad type there its going to see 400,000 distinct values and try to create dummies for all of them. That's bad. Lets look at all our categorical variables and see how many distinct counts there are...

In [75]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
421097
term
2
int_rate
110
grade
7
sub_grade
35
emp_title
120812
emp_length
11
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
url
421095
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
4
last_credit_pull_d
26
application_type
2
verification_status_joint
3


Well that right there is what's called a problem. Some of these have over a hundred thousand distinct types. Lets drop the ones with over 30 unique values, converting to numeric where it makes sense. In doing this there's a lot of code that gets written to just see if the numeric conversion makes sense. It's a manual process that we'll abstract away and just include the conversion.

You could extract numeric features from the dates, but here we'll just drop them. There's a lot of data, it shouldn't be a huge problem.

In [76]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc','id','member_id'], 1, inplace=True)

Wonder what was causing the dtype error on the id column, which _should_ have all been integers? Let's look at the end of the file.

In [77]:
y2015.tail()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
421092,13000.0,13000.0,13000.0,60 months,15.99,316.07,D,5 years,RENT,35000.0,...,0.0,3.0,100.0,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0
421093,12000.0,12000.0,12000.0,60 months,19.99,317.86,E,1 year,RENT,64400.0,...,1.0,2.0,95.0,66.7,0.0,0.0,96919.0,58418.0,9700.0,69919.0
421094,20000.0,20000.0,20000.0,36 months,11.99,664.20,B,10+ years,RENT,100000.0,...,0.0,1.0,100.0,50.0,0.0,1.0,43740.0,33307.0,41700.0,0.0
421095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

Now this should be better. Let's try again.

In [79]:
pd.get_dummies(y2015)

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,...,last_credit_pull_d_Nov-2016,last_credit_pull_d_Oct-2015,last_credit_pull_d_Oct-2016,last_credit_pull_d_Sep-2015,last_credit_pull_d_Sep-2016,application_type_INDIVIDUAL,application_type_JOINT,verification_status_joint_Not Verified,verification_status_joint_Source Verified,verification_status_joint_Verified
0,16000.0,16000.0,16000.0,14.85,379.39,48000.0,33.18,0.0,0.0,33.0,...,0,0,0,0,0,1,0,0,0,0
1,9600.0,9600.0,9600.0,7.49,298.58,60000.0,22.44,0.0,0.0,NaN,...,0,0,0,0,0,1,0,0,0,0
2,25000.0,25000.0,25000.0,7.49,777.55,109000.0,26.02,0.0,1.0,NaN,...,0,0,0,0,0,1,0,0,0,0
3,28000.0,28000.0,28000.0,6.49,858.05,92000.0,21.60,0.0,0.0,42.0,...,0,0,0,0,0,1,0,0,0,0
4,8650.0,8650.0,8650.0,19.89,320.99,55000.0,25.49,0.0,4.0,NaN,...,0,0,0,0,0,1,0,0,0,0
5,23000.0,23000.0,23000.0,8.49,471.77,64000.0,18.28,0.0,0.0,29.0,...,0,0,0,0,0,1,0,0,0,0
6,29900.0,29900.0,29900.0,12.88,678.49,65000.0,21.77,0.0,0.0,NaN,...,0,0,0,0,0,1,0,0,0,0
7,18000.0,18000.0,18000.0,11.99,400.31,112000.0,8.68,0.0,0.0,NaN,...,0,0,0,0,0,1,0,0,0,0
8,35000.0,35000.0,35000.0,14.85,829.90,110000.0,17.06,0.0,0.0,NaN,...,0,0,0,0,0,1,0,0,0,0
9,10400.0,10400.0,10400.0,22.45,289.91,104433.0,25.37,1.0,3.0,12.0,...,0,0,0,0,0,1,0,0,0,0


It finally works! We had to sacrifice sub grade, state address and description, but that's fine. If you want to include them you could run the dummies independently and then append them back to the dataframe.

## Second Attempt

Now let's try this model again.

We're also going to drop NA columns, rather than impute, because our data is rich enough that we can probably get away with it.

This model may take a few minutes to run.

In [82]:
rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']


In [83]:
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

array([0.98093135, 0.98048016, 0.98140628, 0.98173874, 0.97739254,
       0.97744004, 0.96836781, 0.98090669, 0.98021707, 0.98092908])

The score cross validation reports is the accuracy of the tree. Here we're about 98% accurate.

That works pretty well, but there are a few potential problems. Firstly, we didn't really do much in the way of feature selection or model refinement. As such there are a lot of features in there that we don't really need. Some of them are actually quite impressively useless.

There's also some variance in the scores. The fact that one gave us only 93% accuracy while others gave higher than 98 is concerning. This variance could be corrected by increasing the number of estimators. That will make it take even longer to run, however, and it is already quite slow.

## DRILL: Third Attempt

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

In [103]:
#Run PCA on the data as is
sklearn_pca = PCA(n_components=2)
Y_sklearn = sklearn_pca.fit_transform(X)

In [104]:
from sklearn import preprocessing
data_scaled = pd.DataFrame(preprocessing.scale(X),columns = X.columns)

In [105]:
var_list = pd.DataFrame(sklearn_pca.components_,columns=data_scaled.columns,index = ['PC-1','PC-2'])

In [118]:
P1_max =0
P2_max =0

for item in var_list.iteritems():
    P1 = float(item[1][0])
    P2 = float(item[1][1])
    if P1 > P1_max:
        P1_max = P1
        P1_name = item[0]
    if P2 > P2_max:
        P2_name = item[0]
        P2_max = P2

print(P1_name,P1_max,P2_name,P2_max)

tot_hi_cred_lim 0.731710960212435 annual_inc 0.9535904168588791


In [124]:
p1_list=[]
p2_list=[]

for item in var_list.iteritems():
    p1_list.append([str(item[0]),float(item[1][0])])
    p2_list.append([str(item[0]),float(item[1][1])])

In [167]:
sorted(p1_list,key=lambda x: x[1],reverse=True);

In [169]:
sorted(p2_list,key=lambda x: x[1],reverse=True);

In [164]:
#From our lists we can see most of the variance in the dataset is explained by annual_inc
#tot_hi_cred_lim, tot_cur_bal
#Let's keep all rows that are 0.1 or higher of vairance and then we can see if everything
#below 0.5 can be eliminated as well

item_list = []
for item in p2_list:
    if item[1] > 0.015:
        if item not in item_list:
            item_list.append(item[0])
            
for item in p1_list:
    if item[1] > 0.015:
        if item not in item_list:
            item_list.append(item[0])

print(item_list)
print(len(item_list))



['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'annual_inc', 'revol_bal', 'total_pymnt', 'total_pymnt_inv', 'total_rev_hi_lim', 'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit', 'annual_inc', 'revol_bal', 'tot_cur_bal', 'total_rev_hi_lim', 'avg_cur_bal', 'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit']
20


In [165]:
X_short = X.loc[:,item_list]

In [147]:
#With 26 features
#cross_val_score(rfc, X_short, Y, cv=10)

array([0.91358552, 0.96763316, 0.96449859, 0.96129277, 0.9604607 ,
       0.96043695, 0.96041226, 0.96050725, 0.96048163, 0.96048069])

In [160]:
#With 20 features
#cross_val_score(rfc, X_short, Y, cv=10)

array([0.85132151, 0.92194439, 0.9244378 , 0.92059082, 0.91496082,
       0.91165994, 0.91068418, 0.90771569, 0.90517016, 0.89787679])

In [163]:
#With 18 features
#cross_val_score(rfc, X_short, Y, cv=10)

array([0.65087507, 0.65381967, 0.65507825, 0.65552943, 0.65438138,
       0.65236286, 0.65420694, 0.65458691, 0.65276557, 0.65757849])

In [166]:
#20 Features is the winner!
print('Our columns names are:{}'.format(X_short.columns))

Our columns names are:Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'annual_inc',
       'revol_bal', 'total_pymnt', 'total_pymnt_inv', 'total_rev_hi_lim',
       'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit',
       'annual_inc', 'revol_bal', 'tot_cur_bal', 'total_rev_hi_lim',
       'avg_cur_bal', 'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit'],
      dtype='object')
